<a href="https://colab.research.google.com/github/jonkarrer/sales-forecasting-ml/blob/main/Sales_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Store Sales - Time Series Forcasting

In this competition, we will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

In [50]:
from google.colab import userdata
from pathlib import Path
import os
from kaggle import api
import pandas as pd
import numpy as np

Install Kaggle

In [42]:
!pip install kaggle

Sign into Kaggle using ENVs

In [47]:
kaggle_username = userdata.get('KAGGLE_USERNAME')
kaggle_key = userdata.get('KAGGLE_KEY')
creds=f'{{"username":"{kaggle_username}","key":"{kaggle_key}"}}'

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_key

Grab the dataset from Kaggle

In [63]:
COMPETITION_NAME = "store-sales-time-series-forecasting"

if not os.path.exists(COMPETITION_NAME):
    api.competition_download_cli(COMPETITION_NAME)

!kaggle competitions download -c store-sales-time-series-forecasting

# Unzip the dataset
import zipfile
import os

with zipfile.ZipFile(f"{COMPETITION_NAME}.zip", "r") as zip_ref:
    zip_ref.extractall(f"{COMPETITION_NAME}")

data_path = f"{COMPETITION_NAME}/train.csv"
df = pd.read_csv(data_path)
print(df.head())

store-sales-time-series-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
   id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0


## Understanding the Data

In [109]:
tables_in_data = ["train.csv", "oil.csv", "stores.csv", "holidays_events.csv", "transactions.csv", "test.csv", "sample_submission.csv"]
for table in tables_in_data:
    print(table)
    df = pd.read_csv(f"{COMPETITION_NAME}/{table}", low_memory=False)
    print(df.head())

training_df = pd.read_csv(f"{COMPETITION_NAME}/train.csv", low_memory=False)
test_df = pd.read_csv(f"{COMPETITION_NAME}/test.csv", low_memory=False)
oil_df = pd.read_csv(f"{COMPETITION_NAME}/oil.csv", low_memory=False)
stores_df = pd.read_csv(f"{COMPETITION_NAME}/stores.csv", low_memory=False)
holiday_df = pd.read_csv(f"{COMPETITION_NAME}/holidays_events.csv", low_memory=False)
transactions_df = pd.read_csv(f"{COMPETITION_NAME}/transactions.csv", low_memory=False)

train.csv
   id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0
oil.csv
         date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20
stores.csv
   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4
holid

### Training Set

In [85]:
training_df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')

The training data, comprising time series of features id, date, store_nbr, family, and onpromotion as well as the target sales.

- store_nbr identifies the store at which the products are sold.
- family identifies the type of product sold.
- sales gives the total sales for a product family at a particular store at a - given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.
- date is when the sales occured
- id is the row identifier

In [86]:
training_df.nunique()

,0
id,3000888
date,1684
store_nbr,54
family,33
sales,379610
onpromotion,362


It seems that the family and store number are the most crucial aspects of this table.

In [89]:
print("** Store Number")
print(training_df["store_nbr"].unique())

print("** Family")
print(training_df["family"].unique())

** Store Number
[ 1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 23 24 25 26 27 28 29  3 30
 31 32 33 34 35 36 37 38 39  4 40 41 42 43 44 45 46 47 48 49  5 50 51 52
 53 54  6  7  8  9]
** Family
['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']


### Stores

- Store metadata, including city, state, type, and cluster.
- cluster is a grouping of similar stores.
- store_nbr should be what is in the uniques of the training set

In [97]:
stores_df.nunique()

,0
store_nbr,54
city,22
state,16
type,5
cluster,17


In [98]:
print("** Type")
print(stores_df["type"].unique())

print("** Cluster")
print(stores_df["cluster"].unique())

print("** City")
print(stores_df["city"].unique())

print("** State")
print(stores_df["state"].unique())

** Type
['D' 'B' 'C' 'E' 'A']
** Cluster
[13  8  9  4  6 15  7  3 12 16  1 10  2  5 11 14 17]
** City
['Quito' 'Santo Domingo' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil' 'Salinas' 'Daule' 'Babahoyo'
 'Quevedo' 'Playas' 'Libertad' 'Cuenca' 'Loja' 'Machala' 'Esmeraldas'
 'Manta' 'El Carmen']
** State
['Pichincha' 'Santo Domingo de los Tsachilas' 'Cotopaxi' 'Chimborazo'
 'Imbabura' 'Bolivar' 'Pastaza' 'Tungurahua' 'Guayas' 'Santa Elena'
 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas' 'Manabi']


### Oil

Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [92]:
oil_df.nunique()

,0
date,1218
dcoilwtico,998


### Holiday Events

- Holidays and Events, with metadata
- NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [93]:
holiday_df.nunique()

,0
date,312
type,6
locale,3
locale_name,24
description,103
transferred,2


In [94]:
print("** Type")
print(holiday_df["type"].unique())

print("** Locale")
print(holiday_df["locale"].unique())

print("** Locale Name")
print(holiday_df["locale_name"].unique())

print("Transferred")
print(holiday_df["transferred"].unique())

** Type
['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']
** Locale
['Local' 'Regional' 'National']
** Locale Name
['Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda'
 'Imbabura' 'Latacunga' 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe'
 'Esmeraldas' 'Ecuador' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas'
 'Guayaquil']
Transferred
[False  True]


I would like to peak in and see how many holidays are type "transfer" or what is the count of "trasferred" is true

In [110]:
holiday_df["transferred"].value_counts()

,count
transferred,
False,338
True,12


Seeing that the transfer count is so low, I will remove them as an outlier most likely during data prep.

### Transactions

The date, store number, and how many transactions occured that day.

In [99]:
transactions_df.nunique()

,0
date,1682
store_nbr,54
transactions,4993


## Data Preperation

I think ultimatley we want a single table with as much enriched and useful data as possible on it. Our training dataset can enriched with data from our tables in theory.

### Target Metric

This Kaggle competition tells us to use Root Mean Squared Log Error, and our goal is to predict the sales column. So we should go ahead and make the sales column the log of the sales, but we need to handle zero sales. To do this we use the log1p method from numpy.

In [71]:
df = pd.read_csv(f"{COMPETITION_NAME}/train.csv", low_memory=False)

dep_var = "sales"
df[dep_var] = np.log1p(df[dep_var])

### Dates

A common approach for dates is to expand them into day of the week or month and if it is a holiday.

In [111]:
# Remove the transferred holidays from the holiday_events.csv
holiday_df = holiday_df[holiday_df["transferred"] == False]
holiday_df = holiday_df.drop(columns=["transferred", "description"])
holiday_df = holiday_df[holiday_df["type"] != "Transfer"]
holiday_df.head()
# Loop through each value in the "data" column of train.csv
# Lookup the date in the holiday_events.csv
# Apply the type and local_name to train.csv
# Datepart the dates as well

,date,type,locale,locale_name
0,2012-03-02,Holiday,Local,Manta
1,2012-04-01,Holiday,Regional,Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca
3,2012-04-14,Holiday,Local,Libertad
4,2012-04-21,Holiday,Local,Riobamba
